In [3]:
from bse_dsdm.preprocessing.feature_scaling import *
from bse_dsdm.preprocessing.outlier_detection import *
from bse_dsdm.preprocessing.value_encoder import *
from bse_dsdm.preprocessing.value_imputer import *
from bse_dsdm.preprocessing.load_data import *
from bse_dsdm.preprocessing.exploratory_analysis import *
from bse_dsdm.accuracy_testing.cross_validation import *
import pandas as pd
from sklearn.model_selection import train_test_split
from abc import ABC, abstractmethod
import pandas as pd
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc
from sklearn.model_selection import GridSearchCV

LOAD DATA

In [4]:
df=load_csv_data('../train.csv')
print(df)

          id   short_name  overall  potential   value_eur  wage_eur  \
0     216302    E. García       71         71   1400000.0     10000   
1     237867   D. Cancola       65         71   1000000.0      2000   
2     253472      E. Kahl       65         77   1600000.0      2000   
3     223994    S. Mugoša       72         72   2300000.0      5000   
4     251635  A. Țigănașu       65         65    525000.0      3000   
...      ...          ...      ...        ...         ...       ...   
6890  242007    C. Whelan       59         66    425000.0      2000   
6891  187961     Paulinho       83         83  28500000.0     61000   
6892  257234   Y. Hamache       70         80   3400000.0      6000   
6893  232511    S. Sasaki       71         71   1300000.0      7000   
6894  246750    S. Haudum       66         70   1100000.0      4000   

     birthday_date  height_cm  weight_kg                  club_name  ...  \
0       1989-12-28        176         73  Club Atlético de San Luis  ..

In [5]:
df=remove_columns_with_na(df,80)
print(df)


          id   short_name  overall  potential   value_eur  wage_eur  \
0     216302    E. García       71         71   1400000.0     10000   
1     237867   D. Cancola       65         71   1000000.0      2000   
2     253472      E. Kahl       65         77   1600000.0      2000   
3     223994    S. Mugoša       72         72   2300000.0      5000   
4     251635  A. Țigănașu       65         65    525000.0      3000   
...      ...          ...      ...        ...         ...       ...   
6890  242007    C. Whelan       59         66    425000.0      2000   
6891  187961     Paulinho       83         83  28500000.0     61000   
6892  257234   Y. Hamache       70         80   3400000.0      6000   
6893  232511    S. Sasaki       71         71   1300000.0      7000   
6894  246750    S. Haudum       66         70   1100000.0      4000   

     birthday_date  height_cm  weight_kg                  club_name  ...  \
0       1989-12-28        176         73  Club Atlético de San Luis  ..

Preprocessing

In [6]:
categorical_columns_na=columns_with_na(df)[0]
print(categorical_columns_na)
numerical_columns_na=columns_with_na(df)[1]
print(numerical_columns_na)


['club_joined', 'player_traits']
['value_eur', 'release_clause_eur', 'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic']


In [7]:
df=impute_values(df,numerical_columns_na, impute_type='mean')

In [8]:
df=impute_categorical(df,categorical_columns_na)

In [9]:
mapping={'work_rate': {'Low/Low': 1, 'Low/Medium': 2, 'Low/High': 3, 'Medium/Low': 4, 'Medium/Medium': 5, 'Medium/High': 6, 'High/Low': 7, 'High/Medium': 8, 'High/High': 9}}

df = encode_ordinal_columns(df, columns=['work_rate'], mapping=mapping)

In [10]:
df['nationality_name'] = target_encode_column(df, 'nationality_name', 'height_cm', compute_type='mean')

In [11]:
df=encode_categorical_columns(df,column='preferred_foot')

c:\Users\julia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [12]:
df = convert_to_age(df, 'birthday_date',2021)

In [13]:
df=extract_string(df,'body_type')

In [14]:
split_data(df)
test_data,train_data=split_data(df)

In [15]:
feature_columns=['attacking_crossing', 'attacking_finishing', 'attacking_heading_accuracy', 'attacking_short_passing', 'attacking_volleys', 'skill_dribbling', 'skill_curve', 'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control', 'movement_acceleration', 'movement_sprint_speed', 'movement_agility', 'movement_reactions', 'movement_balance', 'power_shot_power', 'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots', 'mentality_aggression', 'mentality_interceptions', 'mentality_positioning', 'mentality_vision', 'mentality_penalties', 'mentality_composure', 'defending_marking_awareness', 'defending_standing_tackle', 'defending_sliding_tackle']
target_column=['position']
X=df[feature_columns]
y=df['position']


In [16]:

model = RandomForestModel(feature_columns, target_column, train_data, test_data)
model.train()
predictions = model.predict()
print(f"predictions:{predictions}")

accuracy = model.get_accuracy(predictions, test_data[target_column])
print(f"Accuracy: {accuracy}")


c:\Users\julia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


predictions:['LM' 'RCB' 'LM' ... 'ST' 'RCB' 'GK']
Accuracy: 0.3665699782451051


In [17]:

model = RandomForestModel(feature_columns, target_column, train_data, test_data)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

In [18]:

model = RandomForestModel(feature_columns, target_column, train_data, test_data)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}


best_model = perform_grid_search(model, param_grid)


Fitting 5 folds for each of 81 candidates, totalling 405 fits


c:\Users\julia\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Best Hyperparameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Best Accuracy: 0.3872490118577075
Test Accuracy with Best Hyperparameters: 0.36276287164612037


In [ ]:
f1= evaluate_best_model(best_model, model.test_data[model.feature_columns], model.test_data[model.target_column])

F1_score: 0.3196409354821298
